# Libraries

In [1]:
#Importar Bibliotecas

import pandas as pd
import numpy as np
import re


import warnings
warnings.filterwarnings("ignore")

 # Documentation

In [2]:
# Importar archivos
df_health = pd.read_csv('../data/HNP_StatsData Alfonso-Copy1.csv')
df_seaice = pd.read_csv('../data/seaice.csv')
df_temp = pd.read_csv('../data/GlobalLandTemperaturesByCountry.csv')
df_temp_change = pd.read_csv('../data/Temperatura_cambio.csv')
df_fires = pd.read_csv('../data/fires.csv')
df_iso = pd.read_csv('../data/iso.csv')
df_agri = pd.read_csv('../data/Agricultura.csv')
df_emissions_origen = pd.read_csv('../data/Emisiones_origen.csv')
df_emisiones = pd.read_csv('../data/owid-co2-data-Copy1.csv')

# Cleaning df_health

In [3]:
#selección de todas las filas que contengan los indicadores que me interesan
df_population = df_health[df_health['Indicator Name'].isin(['Urban population (% of total population)','Urban population growth (annual %)','Rural population (% of total population)', 'Rural population growth (annual %)', 'Population growth (annual %)', 'Population, total'])]
df_health_indicators = df_health[df_health['Indicator Name'].isin(['Treatment for hypertension (% of adults ages 30-79 with hypertension)',\
                                                                   'Total alcohol consumption per capita (liters of pure alcohol, projected estimates, 15+ years of age)',\
                                                                   'Suicide mortality rate (per 100,000 population)','People using at least basic drinking water services (% of population)', \
                                                                   'People using safely managed sanitation services (% of population)',\
                                                                   'Number of surgical procedures (per 100,000 population)',\
                                                                   'Mortality rate attributed to unsafe water, unsafe sanitation and lack of hygiene (per 100,000 population)',\
                                                                   'Mortality rate attributed to household and ambient air pollution (per 100,000 population)',\
                                                                   'Human capital index (HCI) (scale 0-1)', 'External health expenditure (% of current health expenditure)',\
                                                                   'Current health expenditure (% of GDP)', 'Current health expenditure per capita (current US$)',\
                                                                   'Domestic general government health expenditure (% of GDP)', 'Prevalence of overweight (% of adults)'])]


In [4]:
#Eliminar columnas que no me interesan
df_population = df_population.drop(['Unnamed: 66','Indicator Code'], axis = 1)
df_health_indicators = df_health_indicators.drop(['Unnamed: 66','Indicator Code'], axis = 1)

In [5]:
#Reorganizar los datos dejando 'Year' como columna
df_population = df_population.melt(id_vars = ['Country Name', 'Country Code', 'Indicator Name']).rename({'variable': 'Year'}, axis=1 )
df_health_indicators = df_health_indicators.melt(id_vars = ['Country Name', 'Country Code', 'Indicator Name']).rename({'variable': 'Year'}, axis=1 )

In [6]:
#Selección de datos para modelo
df_health_indicators_model = df_health_indicators[(df_health_indicators['Indicator Name'] == 'Mortality rate attributed to household and ambient air pollution (per 100,000 population)') | 
                                            (df_health_indicators['Indicator Name'] == 'Prevalence of overweight (% of adults)') | 
                                            (df_health_indicators['Indicator Name'] == 'Suicide mortality rate (per 100,000 population)') |
                                            (df_health_indicators['Indicator Name'] == 'Number of surgical procedures (per 100,000 population)') | 
                                            (df_health_indicators['Indicator Name'] == 'Treatment for hypertension (% of adults ages 30-79 with hypertension)')]

In [7]:
#Selección de datos para modelo
df_population_model = df_population[(df_population['Indicator Name'] == 'Urban population growth (annual %)') | 
              (df_population['Indicator Name'] == 'Rural population growth (annual %)')]


#### Data Population and health indicators

In [8]:
#Guardar csv
df_population.to_csv('../data/df_population.csv')
df_health_indicators.to_csv('../data/df_health_indicators.csv')
df_health_indicators_model.to_csv('../data/df_health_indicators_model.csv')
df_population_model.to_csv('../data/df_population_model.csv')

# Cleaning df_Seaice

In [9]:
df_seaice.head()

,Year,Month,Day,Extent,Missing,Source Data,hemisphere
0,1978,10,26,10.231,0.0,['ftp://sidads.colorado.edu/pub/DATASETS/nsid...,north
1,1978,10,28,10.420,0.0,['ftp://sidads.colorado.edu/pub/DATASETS/nsid...,north
2,1978,10,30,10.557,0.0,['ftp://sidads.colorado.edu/pub/DATASETS/nsid...,north
3,1978,11,1,10.670,0.0,['ftp://sidads.colorado.edu/pub/DATASETS/nsid...,north
4,1978,11,3,10.777,0.0,['ftp://sidads.colorado.edu/pub/DATASETS/nsid...,north


In [10]:
#Eliminar columnas
df_seaice_model = df_seaice.drop(['    Missing',' Source Data'], axis = 1)

In [11]:
#Eliminar columnas que no me aportan valor
df_seaice = df_seaice.drop(['    Missing', ' Day', ' Month', ' Source Data'], axis = 1)

In [12]:
# Renombrar la columna '     Extent' a 'Extent'
df_seaice = df_seaice.rename(columns={'     Extent': 'Extent'})


In [13]:
#Agrupar por media y año, y reducir la tabla a 1 registro por año
mean_year = df_seaice.groupby('Year')['Extent'].mean()
df_mean_year_ice = pd.DataFrame({'Year': mean_year.index, 'Media': mean_year.values})
df_mean_year_ice = df_mean_year_ice.rename(columns={'Media': 'Ice Extent'})

#### Seaice Mean

In [114]:
#Guardar csv
df_mean_year_ice.to_csv('../data/df_Deshielo_Media.csv')
df_seaice_model.to_csv('../data/df_seaice_model.csv')

# Cleaning df_temp

In [15]:
df_temp.head()

,dt,AverageTemperature,AverageTemperatureUncertainty,Country
0,1743-11-01,4.384,2.294,Åland
1,1743-12-01,NaN,NaN,Åland
2,1744-01-01,NaN,NaN,Åland
3,1744-02-01,NaN,NaN,Åland
4,1744-03-01,NaN,NaN,Åland


In [16]:
# Eliminar los registros por debajo de 1960
df_temp = df_temp[df_temp['dt'].str[:4].astype(int) >= 1960]

In [17]:
# Convertir la columna 'dt' a tipo datetime
df_temp['dt'] = pd.to_datetime(df_temp['dt'])

# Extraer el año de la columna 'dt'
df_temp['Year'] = df_temp['dt'].dt.year

# Eliminar la columna 'dt'
df_temp = df_temp.drop('dt', axis=1)


In [18]:
#Reindexar las columnas
df_temp = df_temp.reindex(columns = ['Year','Country', 'AverageTemperature', 'AverageTemperatureUncertainty' ])

In [19]:
# Agrupar por year y country, encontrar la media para cada año y cada país y asignar valores.
df_temp = df_temp.groupby(['Year', 'Country']).agg({'AverageTemperature': 'mean', 'AverageTemperatureUncertainty': 'mean'}).reset_index()
df_temp = df_temp.rename(columns={'AverageTemperature': 'Average_Temperature'})
df_temp = df_temp.rename(columns={'AverageTemperatureUncertainty': 'Average_Temperature_Uncertainty'})


In [20]:
#Selección para el modelo
df_temp_model = df_temp.drop('Average_Temperature_Uncertainty', axis=1)

In [115]:
#Guardar csv
df_temp.to_csv('../data/df_temp_media.csv')
df_temp_model.to_csv('../data/df_temp_model.csv')

# Cleaning df_temp_change

In [22]:
df_temp_change.head()

,Domain Code,Domain,Area Code (M49),Area,Element Code,Element,Months Code,Months,Year Code,Year,Unit,Value,Flag,Flag Description
0,ET,Temperature change,4,Afghanistan,7271,Temperature change,7020,Meteorological year,1980,1980,°C,0.690,E,Estimated value
1,ET,Temperature change,4,Afghanistan,7271,Temperature change,7020,Meteorological year,1981,1981,°C,0.583,E,Estimated value
2,ET,Temperature change,4,Afghanistan,7271,Temperature change,7020,Meteorological year,1982,1982,°C,-0.237,E,Estimated value
3,ET,Temperature change,4,Afghanistan,7271,Temperature change,7020,Meteorological year,1983,1983,°C,0.213,E,Estimated value
4,ET,Temperature change,4,Afghanistan,7271,Temperature change,7020,Meteorological year,1984,1984,°C,0.210,E,Estimated value


In [23]:
# Eliminar columnas que no me interesan
df_temp_change = df_temp_change.drop(['Unit','Months', 'Domain','Domain Code', 'Area Code (M49)', 'Element Code', 'Months Code', 'Year Code', 'Flag'], axis = 1)

In [24]:
#Reordenar las columnas
df_temp_change = df_temp_change.reindex(columns = ['Year', 'Area', 'Element', 'Value', 'Flag Description'])

In [25]:
#Renombrar las columnas
df_temp_change = df_temp_change.rename(columns = {'Area' : 'Country', 'Flag Description' : 'Description'})

In [26]:
# Añadir etiqueta de valor a value
df_temp_change["Value"] = df_temp_change["Value"].astype(str) + " ºC"


In [27]:
#Guardar csv
df_temp_change.to_csv('../data/df_temp_change.csv')

# Cleaning df_fires

In [28]:
df_fires.head()

,Domain Code,Domain,Area Code (M49),Area,Element Code,Element,Item Code,Item,Year Code,Year,Source Code,Source,Unit,Value,Flag,Flag Description,Note
0,GI,Fires,4,Afghanistan,7246,Burned Area,6796,Humid tropical forest,1990,1990,3050,FAO TIER 1,ha,0.0,E,Estimated value,NaN
1,GI,Fires,4,Afghanistan,7246,Burned Area,6796,Humid tropical forest,1991,1991,3050,FAO TIER 1,ha,0.0,E,Estimated value,NaN
2,GI,Fires,4,Afghanistan,7246,Burned Area,6796,Humid tropical forest,1992,1992,3050,FAO TIER 1,ha,0.0,E,Estimated value,NaN
3,GI,Fires,4,Afghanistan,7246,Burned Area,6796,Humid tropical forest,1993,1993,3050,FAO TIER 1,ha,0.0,E,Estimated value,NaN
4,GI,Fires,4,Afghanistan,7246,Burned Area,6796,Humid tropical forest,1994,1994,3050,FAO TIER 1,ha,0.0,E,Estimated value,NaN


In [29]:
#Eliminar columnas
df_fires = df_fires.drop(['Flag Description', 'Element',  'Domain', 'Area Code (M49)', 'Element Code','Source', 'Unit', 'Flag', 'Item Code', 'Year Code', 'Source Code', 'Note', 'Domain Code'], axis = 1)

In [30]:
# Añadir etiqueta de valor a value
df_fires["Value"] = df_fires["Value"].astype(str) + " Ha"

In [31]:
#Renombrar y reindexar
df_fires = df_fires.rename(columns = {'Area' : 'Country'})
df_fires = df_fires.reindex(columns = ['Year', 'Country', 'Value'])

In [32]:
#Guardar csv
df_fires.to_csv('../data/df_fires.csv')

# Cleaning df_agri

In [33]:
df_agri.head()

,Domain Code,Domain,Area Code (M49),Area,Element Code,Element,Item Code,Item,Year Code,Year,Unit,Value,Flag,Flag Description
0,EL,Land use indicators,4,Afghanistan,7209,Share in Land area,6610,Agricultural land,1980,1980,%,58.34,E,Estimated value
1,EL,Land use indicators,4,Afghanistan,7209,Share in Land area,6610,Agricultural land,1981,1981,%,58.34,E,Estimated value
2,EL,Land use indicators,4,Afghanistan,7209,Share in Land area,6610,Agricultural land,1982,1982,%,58.34,E,Estimated value
3,EL,Land use indicators,4,Afghanistan,7209,Share in Land area,6610,Agricultural land,1983,1983,%,58.34,E,Estimated value
4,EL,Land use indicators,4,Afghanistan,7209,Share in Land area,6610,Agricultural land,1984,1984,%,58.34,E,Estimated value


In [34]:
#ELiminar columnas, renombrarlas, reordenarlas y añadir elementos
df_agri = df_agri.drop(['Domain Code','Element', 'Domain', 'Area Code (M49)', 'Element Code', 'Item Code', 'Year Code', 'Unit', 'Flag', 'Flag Description'], axis = 1)
df_agri = df_agri.rename(columns = {'Area' : 'Country', 'Item' : 'Indicators'})
df_agri['Value'] = df_agri['Value'].astype(str) + ' %'
df_agri = df_agri.reindex(columns = ['Year', 'Country', 'Indicators', 'Value'])


In [35]:
#Seleccionar datos para el modelo
df_agri_model = df_agri[(df_agri['Indicators'] == 'Forest Land') | 
                              (df_agri['Indicators'] == 'Agricultural land') | 
                              (df_agri['Indicators'] == 'Agriculture area under organic agric.')]

In [36]:
#Guardar csv
df_agri.to_csv('../data/df_agri.csv')
df_agri_model.to_csv('../data/df_agri_model.csv')

# Cleaning df_emissions_origen

In [37]:
df_emissions_origen.head()

,Domain Code,Domain,Area Code (M49),Area,Element Code,Element,Item Code,Item,Year Code,Year,Unit,Value,Flag,Flag Description
0,EM,Emissions shares,4,Afghanistan,7264,Emissions Share (CO2),6996,Farm gate,1990,1990,%,72.2771,E,Estimated value
1,EM,Emissions shares,4,Afghanistan,7264,Emissions Share (CO2),6996,Farm gate,1991,1991,%,103.1012,E,Estimated value
2,EM,Emissions shares,4,Afghanistan,7264,Emissions Share (CO2),6996,Farm gate,1992,1992,%,-7.8862,E,Estimated value
3,EM,Emissions shares,4,Afghanistan,7264,Emissions Share (CO2),6996,Farm gate,1993,1993,%,-5.9066,E,Estimated value
4,EM,Emissions shares,4,Afghanistan,7264,Emissions Share (CO2),6996,Farm gate,1994,1994,%,-4.5758,E,Estimated value


In [38]:
#Eliminar columnas
df_emissions_origen = df_emissions_origen.drop(['Domain', 'Domain Code', 'Area Code (M49)', 'Element Code', 'Item Code', 'Unit', 'Year Code', 'Flag', 'Flag Description'], axis = 1)

In [39]:
df_emissions_origen.head()

,Area,Element,Item,Year,Value
0,Afghanistan,Emissions Share (CO2),Farm gate,1990,72.2771
1,Afghanistan,Emissions Share (CO2),Farm gate,1991,103.1012
2,Afghanistan,Emissions Share (CO2),Farm gate,1992,-7.8862
3,Afghanistan,Emissions Share (CO2),Farm gate,1993,-5.9066
4,Afghanistan,Emissions Share (CO2),Farm gate,1994,-4.5758


In [40]:
#Limpiar, renombrar y reindexar mi dataframe
df_emissions_origen = df_emissions_origen.rename(columns = {'Area' : 'Country', 'Item' : 'Indicators'})
df_emissions_origen['Value'] = df_emissions_origen['Value'].astype(str) + ' %'
df_emissions_origen = df_emissions_origen.reindex(columns = ['Year', 'Country', 'Element', 'Indicators','Value'])

In [41]:
df_emissions_origen.head()

,Year,Country,Element,Indicators,Value
0,1990,Afghanistan,Emissions Share (CO2),Farm gate,72.2771 %
1,1991,Afghanistan,Emissions Share (CO2),Farm gate,103.1012 %
2,1992,Afghanistan,Emissions Share (CO2),Farm gate,-7.8862 %
3,1993,Afghanistan,Emissions Share (CO2),Farm gate,-5.9066 %
4,1994,Afghanistan,Emissions Share (CO2),Farm gate,-4.5758 %


In [42]:
#Selección de datos para modelo
df_emissions_origen_model = df_emissions_origen[(df_emissions_origen['Indicators'] == 'Emissions on agricultural land') |
                                                (df_emissions_origen['Indicators'] == 'Energy') | 
                                                (df_emissions_origen['Indicators'] == 'Waste')]

In [43]:
#Guardar csv
df_emissions_origen.to_csv('../data/df_emissions_origen.csv')
df_emissions_origen_model.to_csv('../data/df_emissions_origen_model.csv')

# Cleaning df_emisiones

In [44]:
df_emisiones.head()

,iso_code,country,year,co2,consumption_co2,co2_growth_prct,co2_growth_abs,trade_co2,co2_per_capita,consumption_co2_per_capita,...,ghg_per_capita,methane,methane_per_capita,nitrous_oxide,nitrous_oxide_per_capita,population,gdp,primary_energy_consumption,energy_per_capita,energy_per_gdp
0,AFG,Afghanistan,1949,0.015,NaN,NaN,NaN,NaN,0.002,NaN,...,NaN,NaN,NaN,NaN,NaN,7624058.0,NaN,NaN,NaN,NaN
1,AFG,Afghanistan,1950,0.084,NaN,475.0,0.070,NaN,0.011,NaN,...,NaN,NaN,NaN,NaN,NaN,7752117.0,9.421400e+09,NaN,NaN,NaN
2,AFG,Afghanistan,1951,0.092,NaN,8.7,0.007,NaN,0.012,NaN,...,NaN,NaN,NaN,NaN,NaN,7840151.0,9.692280e+09,NaN,NaN,NaN
3,AFG,Afghanistan,1952,0.092,NaN,0.0,0.000,NaN,0.012,NaN,...,NaN,NaN,NaN,NaN,NaN,7935996.0,1.001732e+10,NaN,NaN,NaN
4,AFG,Afghanistan,1953,0.106,NaN,16.0,0.015,NaN,0.013,NaN,...,NaN,NaN,NaN,NaN,NaN,8039684.0,1.063052e+10,NaN,NaN,NaN


In [45]:
df_emisiones.columns

Index(['iso_code', 'country', 'year', 'co2', 'consumption_co2',
       'co2_growth_prct', 'co2_growth_abs', 'trade_co2', 'co2_per_capita',
       'consumption_co2_per_capita', 'share_global_co2', 'cumulative_co2',
       'share_global_cumulative_co2', 'co2_per_gdp', 'consumption_co2_per_gdp',
       'co2_per_unit_energy', 'coal_co2', 'cement_co2', 'flaring_co2',
       'gas_co2', 'oil_co2', 'other_industry_co2', 'cement_co2_per_capita',
       'coal_co2_per_capita', 'flaring_co2_per_capita', 'gas_co2_per_capita',
       'oil_co2_per_capita', 'other_co2_per_capita', 'trade_co2_share',
       'share_global_cement_co2', 'share_global_coal_co2',
       'share_global_flaring_co2', 'share_global_gas_co2',
       'share_global_oil_co2', 'share_global_other_co2',
       'cumulative_cement_co2', 'cumulative_coal_co2',
       'cumulative_flaring_co2', 'cumulative_gas_co2', 'cumulative_oil_co2',
       'cumulative_other_co2', 'share_global_cumulative_cement_co2',
       'share_global_cumulative_c

In [46]:
df_emisiones = df_emisiones.rename(columns = {'country' : 'Country', 'year' : 'Year'})
df_emisiones = df_emisiones.drop(columns = ['iso_code'])

In [47]:
df_emisiones = df_emisiones.melt(id_vars = ['Country', 'Year']).rename(columns = {'variable' : 'Indicators'})

In [48]:
df_emisiones.head()

,Country,Year,Indicators,value
0,Afghanistan,1949,co2,0.015
1,Afghanistan,1950,co2,0.084
2,Afghanistan,1951,co2,0.092
3,Afghanistan,1952,co2,0.092
4,Afghanistan,1953,co2,0.106


In [49]:
df_emisiones_model = df_emisiones[(df_emisiones['Indicators'] =='co2') | 
                                  (df_emisiones['Indicators'] =='coal_co2') | 
                                  (df_emisiones['Indicators'] =='cement_co2') | 
                                  (df_emisiones['Indicators'] =='flaring_co2') |
                                  (df_emisiones['Indicators'] =='gas_co2') |
                                  (df_emisiones['Indicators'] =='oil_co2') ]

In [50]:
df_emisiones.to_csv('../data/df_emisiones')
df_emisiones_model.to_csv('../data/df_emisiones_model.csv')

https://data.giss.nasa.gov/gistemp/

In [51]:
df = pd.read_csv('../data/Nasa_temperature.csv')

In [52]:
df.head()

,,,,,,,,,,,,,,,,,,Land-Ocean: Global Means
Year,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec,J-D,D-N,DJF,MAM,JJA,SON
1880,-.18,-.24,-.09,-.16,-.10,-.21,-.18,-.10,-.15,-.23,-.22,-.17,-.17,***,***,-.11,-.16,-.20
1881,-.19,-.14,.03,.05,.06,-.18,.00,-.03,-.15,-.22,-.18,-.07,-.09,-.09,-.17,.05,-.07,-.18
1882,.16,.14,.04,-.16,-.14,-.22,-.17,-.08,-.15,-.23,-.17,-.36,-.11,-.09,.08,-.08,-.16,-.18
1883,-.29,-.37,-.12,-.18,-.18,-.07,-.07,-.14,-.22,-.11,-.24,-.11,-.18,-.20,-.34,-.16,-.10,-.19


https://aqicn.org/data-platform/register/

In [53]:
df_1 = pd.read_csv('../data/madrid-air-quality.csv')

In [54]:
df_1

,date,pm25,pm10,o3,no2,so2,co
0,2023/3/1,48,25,42,27,1,
1,2023/3/2,59,29,31,26,1,
2,2023/3/3,62,21,34,19,1,
3,2023/3/4,47,27,26,26,1,
4,2023/3/5,64,25,32,17,1,
...,...,...,...,...,...,...,...
3229,2014/12/31,,47,10,64,10,
3230,2014/10/11,,11,32,11,1,
3231,2013/12/31,,13,20,18,2,
3232,2014/1/26,,13,32,20,2,


In [55]:
df_2 = pd.read_csv('../data/new-york, usa-air-quality.csv')

In [56]:
df_2

,date,pm25,o3,no2,co
0,2023/3/1,43,28,16,
1,2023/3/2,51,27,13,
2,2023/3/3,28,28,9,
3,2023/3/4,38,32,9,
4,2023/3/5,29,28,11,
...,...,...,...,...,...
3311,2022/4/5,,30,11,2
3312,2022/3/2,,29,,
3313,2022/3/3,,25,14,2
3314,2020/3/31,,,8,


In [57]:
df_3 = pd.read_csv('../data/shanghai-air-quality.csv')

In [58]:
df_3

,date,pm25,pm10,o3,no2,so2,co
0,2023/3/1,150,54,37,16,3,5
1,2023/3/2,107,36,29,18,3,4
2,2023/3/3,56,45,34,23,3,4
3,2023/3/4,84,49,48,21,3,4
4,2023/3/5,97,59,66,27,4,5
...,...,...,...,...,...,...,...
3273,2018/12/31,,34,26,13,4,3
3274,2017/9/10,,26,33,16,3,9
3275,2016/3/13,,61,51,13,8,7
3276,2014/12/31,,55,24,19,15,6


http://aidef.apps.eea.europa.eu/

In [59]:
df_4 = pd.read_csv('../data/data.csv')

In [60]:
df_4.head()

,CountryOrTerritory,ReportingYear,UpdateTime,StationLocalId,SamplingPointLocalId,SamplingPoint_Latitude,SamplingPoint_Longitude,Pollutant,AggregationType,Namespace,Unit,BeginPosition,EndPosition,Validity,Verification,DataCoverage,DataCapture,TimeCoverage,AQValue
0,Italy,2013,2020-07-31T07:46:48.663333Z,STA.IT1273A,SPO.IT1273A_10_NDIR_1998-03-02_00:00:00,39.208611,8.390000,Carbon monoxide (air),1 year hour min,IT.ISPRA.AQD,mg.m-3,2013-01-01,2014-01-01,Valid,Verified,90.159817,100.0,90.159817,0.0000
1,Italy,2013,2020-07-31T07:46:48.663333Z,STA.IT1932A,SPO.IT1932A_10_IR-GFC_2008-01-01_00:00:00,40.364440,18.173060,Carbon monoxide (air),1 year hour min,IT.ISPRA.AQD,mg.m-3,2013-01-01,2014-01-01,Valid,Verified,96.324201,100.0,96.324201,0.0100
2,Italy,2013,2020-07-31T07:46:48.663333Z,STA.IT2013A,SPO.IT2013A_10_IR-GFC_2010-09-05_00:00:00,41.638889,13.323333,Carbon monoxide (air),1 year hour min,IT.ISPRA.AQD,mg.m-3,2013-01-01,2014-01-01,Valid,Verified,98.641553,100.0,98.641553,0.0000
3,Lithuania,2013,2020-07-31T07:46:48.663333Z,STA-LT00003,SPO-LT00003_00010_100,54.715279,25.289444,Carbon monoxide (air),1 year hour min,LT.LT-EPA.AQ,mg.m-3,2013-01-01,2014-01-01,Valid,Verified,94.691781,100.0,94.691781,0.0800
4,Poland,2013,2020-07-31T07:46:48.663333Z,STA_PL0098A,SPO_PL0098A_10_001,51.777603,19.452408,Carbon monoxide (air),1 year hour min,PL.CIEP.AQ,mg.m-3,2013-01-01,2014-01-01,Valid,Verified,98.778539,100.0,98.778539,0.1255


In [61]:
df_4 = df_4.drop(['UpdateTime', 'SamplingPointLocalId', 'SamplingPoint_Latitude', 'SamplingPoint_Longitude', 'AggregationType', 'Namespace', 'Verification', 'Validity'],axis = 1)

In [62]:
len(df_4)

1357816

In [63]:
from IPython.display import HTML

html = '<iframe src=\'https://flo.uri.sh/visualisation/11783084/embed\' title=\'Interactive or visual content\' class=\'flourish-embed-iframe\' frameborder=\'0\' scrolling=\'no\' style=\'width:100%;height:600px;\' sandbox=\'allow-same-origin allow-forms allow-scripts allow-downloads allow-popups allow-popups-to-escape-sandbox allow-top-navigation-by-user-activation\'></iframe>'

HTML(html)


In [64]:
html1 = '<iframe id="ep-chart-8a003cc7-fe0b-4740-82e7-5aba738b8c09" src="https://www.epdata.es/embed/8a003cc7-fe0b-4740-82e7-5aba738b8c09/450" style="width: 100%; min-height: 450px; overflow: hidden;" frameborder="0" scrolling="no" height="450"></iframe>'

HTML(html1)


In [65]:
html2 = '<iframe id="ep-chart" src="https://www.epdata.es/embed/f305b9e6-e82c-4a5f-bcd4-540107f66ec8/450" style="width: 100%; min-height: 450px; overflow: hidden;" frameborder="0" scrolling="no" height="450"></iframe>'

HTML(html2)

In [66]:
html3 = '<iframe id="ep-chart-7088de4d-2e2b-4145-ba91-45169773868a" src="https://www.epdata.es/embed/7088de4d-2e2b-4145-ba91-45169773868a/450" style="width: 100%; min-height: 450px; overflow: hidden;" frameborder="0" scrolling="no" height="450"></iframe>'
HTML(html3)

In [67]:
html4 = '<iframe id="ep-chart-9c096ffd-bf8d-4af8-9995-ebe135abe5cc" src="https://www.epdata.es/embed/9c096ffd-bf8d-4af8-9995-ebe135abe5cc/450" style="width: 100%; min-height: 450px; overflow: hidden;" frameborder="0" scrolling="no" height="450"></iframe>'
HTML(html4)

# Cleaning df_gov

In [68]:
df_gov = pd.read_csv('../data/Environmental_Protection_Expenditures.csv')

In [69]:
df_gov.head()

,ObjectId,Country,ISO2,ISO3,Indicator,Source,CTS_Code,CTS_Name,CTS_Full_Descriptor,Unit,...,F2012,F2013,F2014,F2015,F2016,F2017,F2018,F2019,F2020,F2021
0,1,"Afghanistan, Islamic Rep. of",AF,AFG,Expenditure on biodiversity & landscape protec...,"International Monetary Fund (IMF), Statistics ...",GENB_G14,Expenditure on Biodiversity & Landscape Protec...,"Government and Public Sector Finance, Expendit...",Domestic Currency,...,NaN,NaN,NaN,NaN,1.243785e+08,1.258269e+09,NaN,NaN,NaN,NaN
1,2,"Afghanistan, Islamic Rep. of",AF,AFG,Expenditure on biodiversity & landscape protec...,"International Monetary Fund (IMF), Statistics ...",GENB_G14,Expenditure on Biodiversity & Landscape Protec...,"Government and Public Sector Finance, Expendit...",Percent of GDP,...,NaN,NaN,NaN,NaN,9.433491e-03,9.134202e-02,NaN,NaN,NaN,NaN
2,3,"Afghanistan, Islamic Rep. of",AF,AFG,Expenditure on environment protection,"International Monetary Fund (IMF), Statistics ...",GEN_G14,Environmental Protection Expenditures,"Government and Public Sector Finance, Expendit...",Domestic Currency,...,2.846995e+09,3.344665e+09,2.204964e+09,1.122601e+09,1.702933e+09,2.377937e+09,NaN,NaN,NaN,NaN
3,4,"Afghanistan, Islamic Rep. of",AF,AFG,Expenditure on environment protection,"International Monetary Fund (IMF), Statistics ...",GEN_G14,Environmental Protection Expenditures,"Government and Public Sector Finance, Expendit...",Percent of GDP,...,2.754469e-01,2.994793e-01,1.863813e-01,9.152359e-02,1.291590e-01,1.726226e-01,NaN,NaN,NaN,NaN
4,5,"Afghanistan, Islamic Rep. of",AF,AFG,Expenditure on environmental protection n.e.c.,"International Monetary Fund (IMF), Statistics ...",GENO_G14,Expenditure on Environmental Protection Not El...,"Government and Public Sector Finance, Expendit...",Domestic Currency,...,NaN,NaN,NaN,NaN,9.669078e+07,8.846917e+07,NaN,NaN,NaN,NaN


In [70]:
df_gov = df_gov.rename(columns={c: c[1:] for c in df.columns if c.startswith('F')})


In [71]:
df_gov = df_gov.drop(['ObjectId', 'ISO2', 'ISO3', 'Source', 'CTS_Code', 'CTS_Name', 'CTS_Full_Descriptor'], axis = 1)

In [72]:
df_gov = df_gov.melt(id_vars = ['Country', 'Indicator', 'Unit']).rename({'variable': 'Year'}, axis=1 )


In [73]:
df_gov_model = df_gov = df_gov[df_gov['Unit'] == 'Domestic Currency']

In [74]:
df_gov_model['Indicator'].unique()

array(['Expenditure on biodiversity & landscape protection',
       'Expenditure on environment protection',
       'Expenditure on environmental protection n.e.c.',
       'Expenditure on environmental protection R&D',
       'Expenditure on pollution abatement',
       'Expenditure on waste management',
       'Expenditure on waste water management'], dtype=object)

In [75]:
df_gov_model = df_gov_model[(df_gov_model['Indicator'] == 'Expenditure on biodiversity & landscape protection') |
                            (df_gov_model['Indicator'] == 'Expenditure on environment protection') | 
                            (df_gov_model['Indicator'] == 'Expenditure on pollution abatement')]

In [76]:
df_gov_model.to_csv('../data/df_gov_model.csv')

# Cleaning df_dis

In [77]:
df_dis = pd.read_csv('../data/Climate-related_Disasters_Frequency.csv')

In [78]:
df_dis.head()

,ObjectId,Country,ISO2,ISO3,Indicator,Unit,Source,CTS_Code,CTS_Name,CTS_Full_Descriptor,...,F2012,F2013,F2014,F2015,F2016,F2017,F2018,F2019,F2020,F2021
0,1,"Afghanistan, Islamic Rep. of",AF,AFG,"Climate related disasters frequency, Number of...",Number of,"The Emergency Events Database (EM-DAT) , Centr...",ECCD,Climate Related Disasters Frequency,"Environment, Climate Change, Climate Indicator...",...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,1.0
1,2,"Afghanistan, Islamic Rep. of",AF,AFG,"Climate related disasters frequency, Number of...",Number of,"The Emergency Events Database (EM-DAT) , Centr...",ECCD,Climate Related Disasters Frequency,"Environment, Climate Change, Climate Indicator...",...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,"Afghanistan, Islamic Rep. of",AF,AFG,"Climate related disasters frequency, Number of...",Number of,"The Emergency Events Database (EM-DAT) , Centr...",ECCD,Climate Related Disasters Frequency,"Environment, Climate Change, Climate Indicator...",...,6.0,4.0,2.0,1.0,4.0,1.0,3.0,6.0,5.0,2.0
3,4,"Afghanistan, Islamic Rep. of",AF,AFG,"Climate related disasters frequency, Number of...",Number of,"The Emergency Events Database (EM-DAT) , Centr...",ECCD,Climate Related Disasters Frequency,"Environment, Climate Change, Climate Indicator...",...,3.0,1.0,NaN,4.0,NaN,2.0,2.0,1.0,1.0,1.0
4,5,"Afghanistan, Islamic Rep. of",AF,AFG,"Climate related disasters frequency, Number of...",Number of,"The Emergency Events Database (EM-DAT) , Centr...",ECCD,Climate Related Disasters Frequency,"Environment, Climate Change, Climate Indicator...",...,NaN,NaN,1.0,NaN,NaN,2.0,NaN,NaN,1.0,NaN


In [79]:
df_dis = df_dis.rename(columns={c: c[1:] for c in df_dis.columns if c.startswith('F')})


In [80]:
df_dis.head()

,ObjectId,Country,ISO2,ISO3,Indicator,Unit,Source,CTS_Code,CTS_Name,CTS_Full_Descriptor,...,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
0,1,"Afghanistan, Islamic Rep. of",AF,AFG,"Climate related disasters frequency, Number of...",Number of,"The Emergency Events Database (EM-DAT) , Centr...",ECCD,Climate Related Disasters Frequency,"Environment, Climate Change, Climate Indicator...",...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,1.0
1,2,"Afghanistan, Islamic Rep. of",AF,AFG,"Climate related disasters frequency, Number of...",Number of,"The Emergency Events Database (EM-DAT) , Centr...",ECCD,Climate Related Disasters Frequency,"Environment, Climate Change, Climate Indicator...",...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,"Afghanistan, Islamic Rep. of",AF,AFG,"Climate related disasters frequency, Number of...",Number of,"The Emergency Events Database (EM-DAT) , Centr...",ECCD,Climate Related Disasters Frequency,"Environment, Climate Change, Climate Indicator...",...,6.0,4.0,2.0,1.0,4.0,1.0,3.0,6.0,5.0,2.0
3,4,"Afghanistan, Islamic Rep. of",AF,AFG,"Climate related disasters frequency, Number of...",Number of,"The Emergency Events Database (EM-DAT) , Centr...",ECCD,Climate Related Disasters Frequency,"Environment, Climate Change, Climate Indicator...",...,3.0,1.0,NaN,4.0,NaN,2.0,2.0,1.0,1.0,1.0
4,5,"Afghanistan, Islamic Rep. of",AF,AFG,"Climate related disasters frequency, Number of...",Number of,"The Emergency Events Database (EM-DAT) , Centr...",ECCD,Climate Related Disasters Frequency,"Environment, Climate Change, Climate Indicator...",...,NaN,NaN,1.0,NaN,NaN,2.0,NaN,NaN,1.0,NaN


In [81]:
df_dis['Indicator'].unique()

array(['Climate related disasters frequency, Number of Disasters: Drought',
       'Climate related disasters frequency, Number of Disasters: Extreme temperature',
       'Climate related disasters frequency, Number of Disasters: Flood',
       'Climate related disasters frequency, Number of Disasters: Landslide',
       'Climate related disasters frequency, Number of Disasters: Storm',
       'Climate related disasters frequency, Number of Disasters: TOTAL',
       'Climate related disasters frequency, Number of Disasters: Wildfire'],
      dtype=object)

In [82]:
df_dis = df_dis.drop(['ObjectId', 'ISO2', 'ISO3', 'Source', 'CTS_Code', 'CTS_Name', 'CTS_Full_Descriptor','Climate_Influence'], axis = 1)

In [83]:
df_dis = df_dis.melt(id_vars = ['Country', 'Indicator', 'Unit']).rename({'variable': 'Year'}, axis=1 )


In [84]:
df_dis_model = df_dis[df_dis['Indicator'] == 'Climate related disasters frequency, Number of Disasters: TOTAL']

In [85]:
df_dis_model.to_csv('../data/df_dis_model.csv')

# Cleaning df_atmos

In [86]:
df_atmos = pd.read_csv('../data/Atmospheric_CO%E2%82%82_Concentrations.csv')

In [87]:
df_atmos.head()

,ObjectId,Country,ISO2,ISO3,Indicator,Unit,Source,CTS_Code,CTS_Name,CTS_Full_Descriptor,Date,Value
0,1,World,NaN,WLD,Monthly Atmospheric Carbon Dioxide Concentrations,Parts Per Million,"Dr. Pieter Tans, National Oceanic and Atmosphe...",ECCA,Atmospheric Carbon Dioxide Concentrations,"Environment, Climate Change, Climate Indicator...",1958M03,315.70
1,2,World,NaN,WLD,Monthly Atmospheric Carbon Dioxide Concentrations,Parts Per Million,"Dr. Pieter Tans, National Oceanic and Atmosphe...",ECCA,Atmospheric Carbon Dioxide Concentrations,"Environment, Climate Change, Climate Indicator...",1958M04,317.45
2,3,World,NaN,WLD,Monthly Atmospheric Carbon Dioxide Concentrations,Parts Per Million,"Dr. Pieter Tans, National Oceanic and Atmosphe...",ECCA,Atmospheric Carbon Dioxide Concentrations,"Environment, Climate Change, Climate Indicator...",1958M05,317.51
3,4,World,NaN,WLD,Monthly Atmospheric Carbon Dioxide Concentrations,Parts Per Million,"Dr. Pieter Tans, National Oceanic and Atmosphe...",ECCA,Atmospheric Carbon Dioxide Concentrations,"Environment, Climate Change, Climate Indicator...",1958M06,317.24
4,5,World,NaN,WLD,Monthly Atmospheric Carbon Dioxide Concentrations,Parts Per Million,"Dr. Pieter Tans, National Oceanic and Atmosphe...",ECCA,Atmospheric Carbon Dioxide Concentrations,"Environment, Climate Change, Climate Indicator...",1958M07,315.86


In [88]:
# eliminar a partir del cuarto elemento en la columna "Date"
df_atmos["Year"] = df_atmos["Date"].str[:4]

# eliminar la columna "Date" original
df_atmos = df_atmos.drop("Date", axis=1)

In [89]:
df_atmos = df_atmos.drop(['ObjectId', 'ISO2', 'ISO3', 'Source', 'CTS_Code', 'CTS_Name', 'CTS_Full_Descriptor', 'Country'], axis = 1)
df_atmos = df_atmos.reindex(columns = ['Year', 'Indicator', 'Unit', 'Value'])

In [90]:
df_atmos.head()

,Year,Indicator,Unit,Value
0,1958,Monthly Atmospheric Carbon Dioxide Concentrations,Parts Per Million,315.70
1,1958,Monthly Atmospheric Carbon Dioxide Concentrations,Parts Per Million,317.45
2,1958,Monthly Atmospheric Carbon Dioxide Concentrations,Parts Per Million,317.51
3,1958,Monthly Atmospheric Carbon Dioxide Concentrations,Parts Per Million,317.24
4,1958,Monthly Atmospheric Carbon Dioxide Concentrations,Parts Per Million,315.86


In [91]:
df_atmos_model = df_atmos.groupby("Year").mean()

In [92]:
df_atmos_model.head()

,Value
Year,
1958,315.232000
1959,172.460455
1960,158.601667
1961,158.938333
1962,159.355000


In [93]:
df_atmos_model.to_csv('../data/df_atmos_model.csv')

# Cleaning df_ener

In [94]:
df_ener = pd.read_csv('../data/Energy_Transition.csv')

In [95]:
df_ener['Energy_Type'].unique()

array(['Total Renewable', 'Total Non-Renewable'], dtype=object)

In [96]:
df_ener.columns

Index(['ObjectId', 'Country', 'ISO2', 'ISO3', 'Indicator', 'Technology',
       'Energy_Type', 'Unit', 'Source', 'CTS_Name', 'CTS_Code',
       'CTS_Full_Descriptor', 'F2000', 'F2001', 'F2002', 'F2003', 'F2004',
       'F2005', 'F2006', 'F2007', 'F2008', 'F2009', 'F2010', 'F2011', 'F2012',
       'F2013', 'F2014', 'F2015', 'F2016', 'F2017', 'F2018', 'F2019', 'F2020'],
      dtype='object')

In [97]:
df_ener = df_ener.rename(columns={c: c[1:] for c in df_ener.columns if c.startswith('F')})

In [98]:
df_ener = df_ener.drop(['ObjectId', 'ISO2', 'ISO3', 'Source', 'CTS_Code', 'Indicator', 'CTS_Name', 'CTS_Full_Descriptor'], axis = 1)
df_ener = df_ener.melt(id_vars = ['Country', 'Technology', 'Unit', 'Energy_Type']).rename({'variable': 'Year'}, axis=1 )

In [104]:
df_ener = df_ener.reindex(columns = ['Year', 'Country', 'Technology', 'Energy_Type', 'Unit', 'value'])
df_ener_model = df_ener

In [111]:
df_ener_model = df_ener_model[(df_ener_model['Technology'] == 'Fossil fuels' ) | (df_ener_model['Technology'] == 'Nuclear' ) | 
              (df_ener_model['Technology'] == 'Solar energy' ) | (df_ener_model['Technology'] == 'Wind energy' )]

In [113]:
df_ener_model.to_csv('../data/df_ener_model.csv')